In [1]:
import numpy as np
import pandas as pd
import boto3

In [2]:
# Let's use Amazon S3
# The Access Key ID and Secret Access Key are in ~/.aws/credentials
# Region is set in ~/.aws/config
s3 = boto3.resource('s3')

In [3]:
# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

athena-query-bucket-testda24
aws-glue-assets-936480984004-eu-north-1
hacksprint-2023
temp-transformation-rs
vehicle-bucket-alarm


In [4]:
# Print the file names in the bucket
revolvos_bucket = s3.Bucket('hacksprint-2023')
for object in revolvos_bucket.objects.all():
    print(object.key)

Telematics Data Dictionary 2023_02_14.xlsx
Telematics Data Dictionary 2023_02_14.xlsx - SNAP_SHOT_ITEM (1).csv
Vehicle_Fault.csv
Vehicle_Fault_Alarm.csv
Vehicle_Fault_Alarm_Item.csv
Vehicle_Location.csv
Vehicle_Specs.csv


In [ ]:
%%time
# Load one of the files into a dataframe
obj = s3.Object('hacksprint-2023', 'Vehicle_Specs.csv').get()
df = pd.read_csv(obj['Body'], low_memory=False)

In [5]:
%%time
# Load from the local CSV file instead.
df = pd.read_csv('../S3 original bucket/Vehicle_Specs (1).csv', low_memory=False)

CPU times: user 1.47 s, sys: 138 ms, total: 1.61 s
Wall time: 1.61 s


In [6]:
print(df.dtypes)

VEHICLE_ID                int64
VEHICLE_TYPE             object
VEHICLE_SPEC_ID           int64
VEHICLE_MODEL_YEAR        int64
VEHICLE_MODEL            object
VEHICLE_MAKE             object
VEH_VERSION               int64
VEH_DESC                 object
TGW_SERIAL               object
RETAIL_DT                object
PRODUCT_TYPE             object
PRODUCT_CLASS            object
NOTIF_TYPE               object
MQ_MESSAGE_PUT_DT        object
MODEL_FAMILY             object
MAIN_ASSEMBLY_DT         object
LAST_UPDATE_DT           object
INACTIVE_CD             float64
IN_SERVICE_DT            object
IN_PROCESS               object
IMSI                    float64
GROSS_WT_RATING         float64
FINAL_ASSEMBLY_DT        object
FACTORY_INVOICE_DT       object
ENGINE_TYPE              object
ENGINE_MAKE              object
DISPATCH_DT             float64
CREATE_DT                object
COUNTRY_OF_OPERATION     object
CHASSIS_SERIES           object
CHASSIS_NUMBER            int64
dtype: o

In [7]:
print(df.head())

   VEHICLE_ID VEHICLE_TYPE  VEHICLE_SPEC_ID  VEHICLE_MODEL_YEAR VEHICLE_MODEL  \
0     2494778            D          3379851                2009           VNL   
1     2472691            D          3379743                2008           VNL   
2   100012582            D          3379751                2011         GU713   
3   100237223            D          3010560                2012        CHU613   
4   100183496            D          3008550                2016        CHU613   

  VEHICLE_MAKE  VEH_VERSION               VEH_DESC TGW_SERIAL   RETAIL_DT  \
0        VOLVO           28                Unknown        NaN    6/2/2008   
1        VOLVO           32                Unknown   06009825   1/10/2008   
2         MACK           15  SELECT AN APPLICATION   10500612   6/25/2010   
3         MACK           22                Unknown        NaN  12/31/2011   
4         MACK           90     AGRICULTURE ON-HWY   15072839   6/18/2015   

   ... GROSS_WT_RATING FINAL_ASSEMBLY_DT FACTORY_I

In [8]:
print(df.shape)

(458948, 31)


Insert magical cleaning...

In [ ]:
%%time
# Send the cleaned dataframe content back to a new CSV file on the S3 bucket
csv_clean = df.to_csv(index=False)
_ = s3.Object('hacksprint-2023', 'Vehicle_Specs-clean.csv').put(Body=csv_clean)